In [22]:
# Deeplearning
# Total crime incident prediction using ANN Algorithm
import tensorflow
import keras
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import confusion_matrix
import seaborn as sns

#import dataset
data=pd.read_csv(r"C:\Users\donad\OneDrive\Desktop\Internship\3_DL\Datasets\BPD_Part_1_Victim_Based_Crime_Data (2).csv")
data

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Location 1,Total Incidents
0,11/12/2016,02:35:00,3B,300 SAINT PAUL PL,ROBBERY - STREET,O,NaN,111.0,CENTRAL,Downtown,"(39.2924100000, -76.6140800000)",1
1,11/12/2016,02:56:00,3CF,800 S BROADWAY,ROBBERY - COMMERCIAL,I,FIREARM,213.0,SOUTHEASTERN,Fells Point,"(39.2824200000, -76.5928800000)",1
2,11/12/2016,03:00:00,6D,1500 PENTWOOD RD,LARCENY FROM AUTO,O,NaN,413.0,NORTHEASTERN,Stonewood-Pentwood-Winston,"(39.3480500000, -76.5883400000)",1
3,11/12/2016,03:00:00,6D,6600 MILTON LN,LARCENY FROM AUTO,O,NaN,424.0,NORTHEASTERN,Westfield,"(39.3626300000, -76.5516100000)",1
4,11/12/2016,03:00:00,6E,300 W BALTIMORE ST,LARCENY,O,NaN,111.0,CENTRAL,Downtown,"(39.2893800000, -76.6197100000)",1
...,...,...,...,...,...,...,...,...,...,...,...,...
285802,01/01/2011,22:15:00,4D,6800 MCCLEAN BD,AGG. ASSAULT,I,HANDS,423.0,NORTHEASTERN,Hamilton Hills,"(39.3704700000, -76.5670500000)",1
285803,01/01/2011,22:30:00,6J,3000 ODONNELL ST,LARCENY,I,NaN,232.0,SOUTHEASTERN,Canton,"(39.2804600000, -76.5727300000)",1
285804,01/01/2011,23:00:00,7A,2500 ARUNAH AV,AUTO THEFT,O,NaN,721.0,WESTERN,Evergreen Lawn,"(39.2954200000, -76.6592800000)",1
285805,01/01/2011,23:25:00,4E,100 N MONROE ST,COMMON ASSAULT,I,HANDS,714.0,WESTERN,Penrose/Fayette Street Outreach,"(39.2899900000, -76.6470700000)",1


In [2]:
# checking null variables
data.isnull().sum()

CrimeDate               0
CrimeTime               0
CrimeCode               0
Location             1623
Description             0
Inside/Outside       4196
Weapon             188411
Post                  191
District               58
Neighborhood         1701
Location 1           1619
Total Incidents         0
dtype: int64

In [3]:
# Define columns with missing values
columns_with_missing = ['Location', 'Inside/Outside', 'Weapon', 'Post', 'District', 'Neighborhood']

# Initialize LabelEncoder
label_encoders = {}

# Encode categorical columns
for col in columns_with_missing:
    label_encoders[col] = LabelEncoder()
    data[col] = label_encoders[col].fit_transform(data[col].astype(str))

In [4]:
# Initialize KNN imputer
imputer = KNNImputer(n_neighbors=5, weights='uniform')

# Impute missing values
data[columns_with_missing] = imputer.fit_transform(data[columns_with_missing])

# For Location 1 column, fill with a default value like '(0.0, 0.0)'
data['Location 1'].fillna('(0.0, 0.0)', inplace=True)

In [5]:
# Checking null variables again to ensure no missing values
print(data.isnull().sum())

CrimeDate          0
CrimeTime          0
CrimeCode          0
Location           0
Description        0
Inside/Outside     0
Weapon             0
Post               0
District           0
Neighborhood       0
Location 1         0
Total Incidents    0
dtype: int64


In [6]:
data

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Location 1,Total Incidents
0,11/12/2016,02:35:00,3B,10870.0,ROBBERY - STREET,2.0,4.0,13.0,0.0,62.0,"(39.2924100000, -76.6140800000)",1
1,11/12/2016,02:56:00,3CF,23197.0,ROBBERY - COMMERCIAL,0.0,0.0,36.0,8.0,84.0,"(39.2824200000, -76.5928800000)",1
2,11/12/2016,03:00:00,6D,4189.0,LARCENY FROM AUTO,2.0,4.0,70.0,4.0,240.0,"(39.3480500000, -76.5883400000)",1
3,11/12/2016,03:00:00,6D,21967.0,LARCENY FROM AUTO,2.0,4.0,76.0,4.0,263.0,"(39.3626300000, -76.5516100000)",1
4,11/12/2016,03:00:00,6E,10908.0,LARCENY,2.0,4.0,13.0,0.0,62.0,"(39.2893800000, -76.6197100000)",1
...,...,...,...,...,...,...,...,...,...,...,...,...
285802,01/01/2011,22:15:00,4D,22100.0,AGG. ASSAULT,0.0,1.0,75.0,4.0,106.0,"(39.3704700000, -76.5670500000)",1
285803,01/01/2011,22:30:00,6J,11224.0,LARCENY,0.0,4.0,45.0,8.0,32.0,"(39.2804600000, -76.5727300000)",1
285804,01/01/2011,23:00:00,7A,8375.0,AUTO THEFT,2.0,4.0,137.0,12.0,78.0,"(39.2954200000, -76.6592800000)",1
285805,01/01/2011,23:25:00,4E,1079.0,COMMON ASSAULT,0.0,1.0,135.0,12.0,203.0,"(39.2899900000, -76.6470700000)",1


In [7]:
# encoding categorical variables to numerics
le=LabelEncoder()
dummy=['CrimeDate','CrimeTime','CrimeCode','Description','Location 1']
for i in dummy:
    data[i]=le.fit_transform(data[i].astype(str))
data

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Location 1,Total Incidents
0,1897,491,11,10870.0,13,2.0,4.0,13.0,0.0,62.0,29293,1
1,1897,531,13,23197.0,11,0.0,0.0,36.0,8.0,84.0,14975,1
2,1897,585,53,4189.0,8,2.0,4.0,70.0,4.0,240.0,84254,1
3,1897,585,53,21967.0,8,2.0,4.0,76.0,4.0,263.0,93383,1
4,1897,585,54,10908.0,7,2.0,4.0,13.0,0.0,62.0,25199,1
...,...,...,...,...,...,...,...,...,...,...,...,...
285802,0,3934,41,22100.0,0,0.0,1.0,75.0,4.0,106.0,97086,1
285803,0,3966,58,11224.0,7,0.0,4.0,45.0,8.0,32.0,12655,1
285804,0,4096,61,8375.0,3,2.0,4.0,137.0,12.0,78.0,33895,1
285805,0,4155,42,1079.0,5,0.0,1.0,135.0,12.0,203.0,25981,1


In [8]:
# checking duplicates
data.duplicated().sum()

11403

In [9]:
#droping duplicated rows
data.drop_duplicates(inplace=True)
data.duplicated().sum()

0

In [10]:
data

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Location 1,Total Incidents
0,1897,491,11,10870.0,13,2.0,4.0,13.0,0.0,62.0,29293,1
1,1897,531,13,23197.0,11,0.0,0.0,36.0,8.0,84.0,14975,1
2,1897,585,53,4189.0,8,2.0,4.0,70.0,4.0,240.0,84254,1
3,1897,585,53,21967.0,8,2.0,4.0,76.0,4.0,263.0,93383,1
4,1897,585,54,10908.0,7,2.0,4.0,13.0,0.0,62.0,25199,1
...,...,...,...,...,...,...,...,...,...,...,...,...
285802,0,3934,41,22100.0,0,0.0,1.0,75.0,4.0,106.0,97086,1
285803,0,3966,58,11224.0,7,0.0,4.0,45.0,8.0,32.0,12655,1
285804,0,4096,61,8375.0,3,2.0,4.0,137.0,12.0,78.0,33895,1
285805,0,4155,42,1079.0,5,0.0,1.0,135.0,12.0,203.0,25981,1


In [11]:
# Splittong dataset
x=data.iloc[:,:11].values
y=data.iloc[:,11].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [12]:
#Feature Scaling
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
X_test = sc.transform(x_test)

In [13]:
# Initialising the ANN
classifier = Sequential()

In [14]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(10, kernel_initializer ='uniform', activation='relu', input_dim=11))

# Adding dropout to prevent overfitting
classifier.add(Dropout(rate=0.1))

# Adding the second hidden layer
classifier.add(Dense(10, kernel_initializer ='uniform', activation='relu'))

# Adding dropout to prevent overfitting
classifier.add(Dropout(rate=0.1))

# Adding the output layer
classifier.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

In [15]:
# Compiling the ANN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(x_train, y_train, batch_size=100, epochs=10)


Epoch 1/10


2196/2196 [==============================] - 4s 2ms/step - loss: 0.0392 - accuracy: 0.9999
Epoch 2/10
2196/2196 [==============================] - 3s 2ms/step - loss: 1.4411e-04 - accuracy: 1.0000
Epoch 3/10
2196/2196 [==============================] - 3s 2ms/step - loss: 3.6204e-05 - accuracy: 1.0000
Epoch 4/10
2196/2196 [==============================] - 3s 2ms/step - loss: 1.4287e-05 - accuracy: 1.0000
Epoch 5/10
2196/2196 [==============================] - 3s 2ms/step - loss: 4.8740e-06 - accuracy: 1.0000
Epoch 6/10
2196/2196 [==============================] - 3s 2ms/step - loss: 1.4368e-06 - accuracy: 1.0000
Epoch 7/10
2196/2196 [==============================] - 3s 2ms/step - loss: 1.0141e-06 - accuracy: 1.0000
Epoch 8/10
2196/2196 [==============================] - 3s 2ms/step - loss: 1.6778e-07 - accuracy: 1.0000
Epoch 9/10
2196/2196 [==============================] - 3s 1ms/step - loss: 4.8818e-07 - accuracy: 1.0000
Epoch 10/10
2196/2196 [========================

In [16]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
y_pred

1716/1716 [==============================] - 2s 993us/step


array([[ True],
       [ True],
       [ True],
       ...,
       [ True],
       [ True],
       [ True]])

In [17]:
# accuracy calculation 
accuracy=np.mean(y_pred==y_test)
accuracy

1.0